In [1]:
import polars as pl

df = pl.read_csv("fever_test.filtered.sampled.csv")
df.head()

id,cid,premise,hypothesis,alternative hypothesis,label,new hypothesis,new label,change type
i64,i64,str,str,str,str,str,str,str
0,58846,"""Johnny Galecki . He is known f…","""Johnny Galecki has been in at …","""""","""NEUTRAL""","""""","""""",""""""
1,172460,"""Matteo Renzi ( [ matˈtɛːo ˈrɛn…","""Matteo Renzi served as Prime M…","""""","""ENTAILMENT""","""""","""""",""""""
2,181201,"""Southpaw is a 2015 American sp…","""Southpaw was released 2011.""","""""","""CONTRADICTION""","""""","""""",""""""
3,174024,"""Pink Floyd . The final Pink Fl…","""The Endless River is Pink Floy…","""""","""CONTRADICTION""","""""","""""",""""""
4,118068,"""Beatles Day . This day is cons…","""Liverpool is unrelated to The …","""""","""CONTRADICTION""","""""","""""",""""""


In [26]:
to_skip = df.select('cid').to_numpy()
print(to_skip)

[[ 58846]
 [172460]
 [181201]
 [174024]
 [118068]
 [ 89681]
 [151483]
 [ 69783]
 [207341]
 [184051]
 [214252]
 [173669]
 [ 64105]
 [171511]
 [ 43483]
 [156106]
 [ 74311]
 [  9532]
 [101434]
 [217390]
 [188645]
 [ 26804]
 [ 77007]
 [227791]
 [140462]
 [206021]
 [144725]
 [  7962]
 [165000]
 [ 12612]
 [ 36630]
 [ 53177]
 [225303]
 [187784]
 [216351]
 [208440]
 [100824]
 [114567]
 [152634]
 [ 47607]
 [ 18222]
 [ 28201]
 [ 90991]
 [ 86598]
 [ 84669]
 [104249]
 [208429]
 [ 12264]
 [180731]
 [174520]
 [ 71246]
 [114912]
 [122148]
 [ 10435]
 [110749]
 [ 96181]
 [113664]
 [111707]
 [ 65017]
 [172323]
 [101608]
 [ 53760]
 [128837]
 [ 24204]
 [ 82745]
 [169317]
 [  5397]
 [179268]
 [134850]
 [165872]
 [203996]
 [154698]
 [168545]
 [  7758]
 [173510]
 [189437]
 [225288]
 [ 69399]
 [182042]
 [205741]
 [177974]
 [119207]
 [ 52836]
 [ 56574]
 [178893]
 [119530]
 [ 21933]
 [ 57719]
 [165711]
 [187554]
 [225273]
 [155555]
 [108203]
 [174629]
 [165891]
 [228077]
 [  7202]
 [ 30333]
 [ 90529]
 [ 29143]


In [17]:
from datasets import load_dataset

dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", download_mode="force_redownload")
print(dataset['test'])

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
    num_rows: 2287
})


In [27]:
to_keep = {
    'cid':[], 
    'premise':[], 
    'hypothesis': [], 
    'label': []
}
from tqdm import tqdm
for split in ['test', 'validation']:
    for sample in tqdm(dataset[split], desc=f"> processing {split} data"):
        print(sample['id'])
        if sample['premise'] == '' or sample['hypothesis'] == '': #or sample['id'] in to_skip:
            continue
        if sample['id'] in to_skip:
            print("Easaaf")
        to_keep['cid'].append(sample['id'])
        to_keep['premise'].append(sample['premise'])
        to_keep['hypothesis'].append(sample['hypothesis'])
        to_keep['label'].append(sample['label'])

to_keep_df = pl.from_dict(to_keep)
to_keep_df.height

> processing test data:   8%|▊         | 175/2287 [00:00<00:02, 958.64it/s]

89891
194372
110177
166846
187000
201090
132134
71126
66638
114567
185193
38125
128229
141072
63038
217212
227080
129752
105352
84187
94038
23845
54142
19501
116417
59113
124540
205739
128538
36242
166633
10153
155955
151747
174580
6113
79169
36092
7179
181977
194026
138501
157576
131931
111769
168976
55294
141344
189448
112935
195060
195823
139250
166506
159716
207264
227140
155992
171627
82608
79322
16783
175941
98806
37050
67643
10460
86543
195038
148799
141294
161566
171635
73080
73226
190170
143075
45565
155920
181614
161841
182289
149173
140589
187104
222034
13076
139317
88875
32940
192839
226146
63252
125437
74540
61795
11497
124400
40124
130703
20926
85172
32815
192976
172279
116174
204440
117409
132218
218361
199743
165110
194925
68573
21314
101839
148356
154339
84223
223664
3059
97091
103745
6680
32700
76552
82838
134420
204026
45439
172472
115234
184060
165113
123833
49097
209869
142951
92090
66005
196755
98008
88847
135572
106044
108481
200278
126814
180732
164986
226860
14

> processing test data:  21%|██▏       | 490/2287 [00:00<00:01, 1363.47it/s]

65269
89063
114929
202782
4179
97927
142286
14915
196967
196746
181192
174027
115283
200259
422
83684
29193
65940
44552
1077
124598
110242
106722
218469
80001
216352
69699
203162
189437
203186
202055
103118
96726
76047
133980
155613
159078
121453
23906
97275
121634
38587
18574
63444
77304
13764
185406
192835
23634
14510
175472
217684
208422
2414
179340
156609
99495
46383
100359
128642
36170
22633
78282
173511
137429
227072
13432
36371
161582
32827
106481
88429
135805
85649
124872
58764
183604
29443
59380
59777
161574
52188
32890
83258
105157
216355
195073
167471
172092
116391
64498
123822
194789
77378
107308
91858
206161
4818
224979
48880
79072
47239
185417
66633
80693
11002
144978
203614
113024
75877
81518
227135
85873
195824
143895
200279
78849
214253
162895
183601
17094
49544
175738
34319
202026
39078
225234
227355
131193
107510
195920
115173
202936
16693
55055
11637
153299
82973
87798
11404
13396
16579
194466
14448
178335
8584
197388
31770
19310
54460
215209
95935
143488
189768
275

> processing test data:  34%|███▍      | 786/2287 [00:00<00:01, 1430.35it/s]

...

> processing validation data:  96%|█████████▌| 2201/2288 [00:01<00:00, 1438.31it/s]

62985
28332
19242
183494
64909
153263
69507
33334
143684
182678
113686
161709
154439
27806
88694
50799
204544
37325
168923
150168
199677
103583
79881
94321
9824


> processing validation data: 100%|██████████| 2288/2288 [00:01<00:00, 1433.68it/s]

83989
65702
90865
10455
74806
202311
197659
144498
304
126947
205080
122706
208898
10451
37739
94896
175868
209355
195199
119867
205074
28882
215279
131650
194177
61475
137678
129531
86904
169330
19951
188638
117829
33739
63949
141758
44899
209098
175459
101125
23448
72426
129270
89681
95682
140527
15504
132966
84154
75621
137390
181259
178912
49543
179100
22347
51667
14972
48466
88480
15341
93974
117283
136214
95183
164529
47210
150099
100615
66980
130007
189861
106325
187532
97183
4675
188565
28833
110794
181254
151206
146453
112180
58618
52039
144200
225729


4575